## Land Surface Temperature

- **Install packages and Authenticate to Earth Engine**

In [ ]:
!pip install earthengine-api #Earth-engine Python API
!pip install geopandas  #needed for shapefile import

In [ ]:
!earthengine authenticate



```
# This is formatted as code
```

**Authenticate to Google Drive. We need this for exporting rasters**


In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

In [4]:
# import packages
import os
import geopandas as gpd
import time
from random import randint
import ee

#Initialize EE
ee.Initialize()

**Set paths**


In [ ]:
main_dir=r"/content/drive/My Drive/az/GEE_main_folder"
cities = r"{}/Inputs/Nurpur_shapes.shp".format(main_dir) # path to the cities shapfile that has been uploaded to Gdrive
cities_reprojected = gpd.read_file(cities).to_crs({'init': 'epsg:4326'})

**Define function to export LST rasters**


In [6]:
def export_landsurface_temperature( country , cities_reprojected, max_value , years, month0 ,  month1):

    """
    export_landsurface_temperature: Exports Land Surface temperature rasters for a given country in the cities shapefile

    :param country: Name of the country
    :param cities_reprojected: geopandas read cities shapefile
    :param max_value: cap the number of cities
    :param years: years
    :param month0 : first hottest month
    :param month1 : last hottest month

    :return: confirms the export of rasters

    """

    # Create a jsondictionary from the geometry of the  shapefile
    geom = cities_reprojected['geometry']
    jsonDict = eval(geom.to_json())

    for index, row in cities_reprojected.iterrows():
        try:
          city= row['MAUZA'].lower() #instruct the column for city name in the city shapefile
          if index  <= max_value:
              city_number= index
              for x in jsonDict['features'][city_number]['geometry']['coordinates']:
                  AOI = ee.Geometry.Polygon(x) #cast polygon as ee geometry
                  landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

                  # Filter for hottest months in the past X years
                  def filter_hot_month(i):
                      return ee.Filter.date(years[i] + '-' + month0 + '-01', years[i] + '-' + month1 + '-01')

                  range_list= list(map(filter_hot_month, list(range(0, 10)))) #combination of months and years
                  rangefilter = ee.Filter.Or(range_list)

                  # Define a function to scale the data and mask unwanted pixels
                  def maskL457sr(image):
                      # Bit 0 - Fill
                      # Bit 1 - Dilated Cloud
                      # Bit 2 - Cirrus (high confidence)
                      # Bit 3 - Cloud
                      # Bit 4 - Cloud Shadow
                      qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
                      saturationMask = image.select('QA_RADSAT').eq(0)
                      # Apply the scaling factors to the appropriate bands.
                      thermalBand = image.select('ST_B10').multiply(0.00341802).add(149.0)
                      # Replace the original bands with the scaled ones and apply the masks.
                      return image.addBands(thermalBand, None, True).updateMask(qaMask).updateMask(saturationMask)

                  # Apply filter and mask
                  collectionSummer = landsat.filter(rangefilter).filterBounds(AOI).map(maskL457sr).select('ST_B10').mean().add(-273.15).clip(AOI)
                  task = ee.batch.Export.image.toDrive(**{
                      'image': collectionSummer,
                      'description': city + "_Summer",
                      'folder': country,
                      'region': AOI,
                      'scale': 30,
                      'maxPixels': 1e9
                  })
                  task.start()

        except Exception as e:
            print(f"Error : {e} ")
    return  print(f"Successfully exported the rasters for: {country}")

**Set Args and call the function**


In [8]:
#Args
max_value=5
years =list(str(i) for i in range(2013, 2024)) #years from 2013-2023
month0 = '02'  # first hottest month  # update for each country
month1 = '05'  # end of hottest month (note that this is exclusive)  # update for each country
country= 'DRC' # replace with the relevent country name
start_exporting= export_landsurface_temperature(country, cities_reprojected, max_value, years, month0, month1)

Successfully exported the rasters for: DRC
